# Distillation of gpt 4o mini subquestion and subanswer into llama 3.1 8B for Stock Price Prediction: LLM-Based Forecasting (with Risk-Aware PPO Adjustment)

This notebook runs inference using llama 3.1 8B with distilled inference from teacher model gpt 4o mini (using socratic cot knowledge distillation)

## Framework Overview:
1. **Stage 1**: LLM-based stock price prediction using historical data, technical indicators, and sentiment analysis
2. **Stage 2**: Risk-aware PPO adjustment incorporating VaR and CVaR to refine predictions (ablation from paper)

## Dataset:
- Training, validation, and test data from finetune_paper directory
- Stocks: AAPL, HSBC, PEP, 0700.HK (Tencent), 7203.T (Toyota)

## 1. Environment Setup and Dependencies

In [22]:
# Install required packages (run once)
!pip install -r ../requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [23]:
# Install Hugging Face packages (run once if using local Llama)
!pip install transformers accelerate bitsandbytes torch

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [24]:
# Import libraries
import os
import json
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

# Standard library
import time
import pickle

# Environment variables
from dotenv import load_dotenv

# HTTP requests for HF endpoint
import requests

# # Machine Learning
# from sklearn.svm import SVR
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
# from xgboost import XGBRegressor

# Deep Learning
import torch
import torch.nn as nn

# Reinforcement Learning
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# Progress bar
from tqdm import tqdm

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

print("All libraries imported successfully!")


All libraries imported successfully!


## 2. Hugging Face Dedicated Endpoint Configuration

In [25]:
# Load environment variables
load_dotenv('../.env')

# LLM Configuration
MAX_TOKENS = 1024
TEMPERATURE = 0.0

# Hugging Face Dedicated Endpoint
HF_ENDPOINT_URL = "https://bvpku716v4ri7f76.us-east-1.aws.endpoints.huggingface.cloud"

# Get HF token
hf_token = os.getenv('HF_TOKEN')
if not hf_token:
    raise ValueError("HF_TOKEN not found in .env file. Get token from: https://huggingface.co/settings/tokens")

print(f"✅ Hugging Face Dedicated Endpoint configured!")
print(f"   Endpoint: {HF_ENDPOINT_URL}")
print(f"   Model: Llama 3.1 8B Instruct")
print(f"   Max Tokens: {MAX_TOKENS}")
print(f"   Temperature: {TEMPERATURE}")
print(f"   Rate limits: UNLIMITED! 🎉")

✅ Hugging Face Dedicated Endpoint configured!
   Endpoint: https://bvpku716v4ri7f76.us-east-1.aws.endpoints.huggingface.cloud
   Model: Llama 3.1 8B Instruct
   Max Tokens: 1024
   Temperature: 0.0
   Rate limits: UNLIMITED! 🎉


## 3. Data Loading and Preprocessing

In [26]:
# Load datasets
def load_jsonl(filepath):
    """Load JSONL file"""
    data = []
    with open(filepath, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

# Load train, val, test data
# train_data = load_jsonl('../finetune_paper/train.jsonl')
# val_data = load_jsonl('../finetune_paper/val.jsonl')
test_data = load_jsonl('../finetune_paper/test.jsonl')

# Load supervised labels
all_labels = pd.read_csv('../finetune_paper/all_supervised_price_labels.csv')

# print(f"Training samples: {len(train_data)}")
# print(f"Validation samples: {len(val_data)}")
print(f"Test samples: {len(test_data)}")
print(f"\nAll labels shape: {all_labels.shape}")
print(f"\nStocks in dataset: {all_labels['ticker'].unique()}")

Test samples: 2477

All labels shape: (12418, 16)

Stocks in dataset: ['AAPL' 'HSBC' '0700.HK' 'PEP' '7203.T']


In [27]:
# Display sample data
print("Sample training data:")
print(f"Prompt (first 500 chars): {test_data[0]['prompt']}...")
print(f"\nResponse: {test_data[0]['response']}")

print("\n" + "="*80 + "\n")
print("Sample supervised labels:")
all_labels.head()

Sample training data:
Prompt (first 500 chars): You are a financial analyst with expertise in stock market forecasting.
Your task is to analyze market data and predict the next trading day stock price.
Use historical price trends, technical indicators, and sentiment analysis to provide an informed forecast.
Ensure that your predictions are well-justified, considering multiple financial factors.

• Predicted Stock Price: The forecasted close price for the next trading day.
• Price Movement Likelihood: The likelihood of the predicted stock price.
• Justification: Provide an explanation for the predicted stock price and the corresponding likelihood, considering the following:
  - Historical market data (e.g., recent closing prices).
  - Technical indicators (e.g., SMA, EMA, RSI, MACD, Bollinger Bands).
  - Sentiment analysis (e.g., news sentiment, market sentiment).

Please weigh these signals and justify the predicted stock price.

TICKER: HSBC
DATE: 2023-01-03

RECENT CLOSING PRICES (mo

,Date,SMA_20,SMA_50,EMA_12,EMA_26,RSI_14,MACD,MACD_signal,MACD_hist,BB_width_20_2,headline_count,sent_compound_mean,titles_joined,next_close,confidence_proxy,ticker
0,2015-01-16 00:00:00+00:00,NaN,NaN,27.159062,27.234398,13.536208,-0.075335,-0.015690,-0.059645,NaN,4.0,-0.079550,NaN,27.180000,0.5,AAPL
1,2015-01-16 00:00:00+00:00,NaN,NaN,45.765558,46.231136,4.645025,-0.465578,-0.348537,-0.117041,NaN,6.0,0.308567,Which London business pays the highest busines...,45.360001,0.9,HSBC
2,2015-01-16 00:00:00+00:00,NaN,NaN,113.078837,109.846862,68.406756,3.231975,2.607665,0.624309,NaN,1.0,0.000000,NaN,113.388344,0.5,0700.HK
3,2015-01-16 00:00:00+00:00,NaN,NaN,96.059458,95.400737,36.546590,0.658721,0.411460,0.247261,NaN,10.0,0.082980,"Audrey P. ""Pep"" Landry Obituary January 16, 20...",97.510002,0.5,PEP
4,2015-01-19 00:00:00+00:00,NaN,NaN,113.126453,110.109194,70.079261,3.017259,2.689584,0.327675,NaN,1.0,0.361200,WeChat apologizes for showering Chinese users ...,114.402382,0.5,0700.HK


In [28]:
# Parse test data for evaluation
POSITIVE_JUSTIFICATION_KEYWORDS = {
    "increase", "growth", "upward", "bullish", "positive", "gain", "improve", "strength", "rally", "optimistic"
}
NEGATIVE_JUSTIFICATION_KEYWORDS = {
    "decrease", "decline", "downward", "bearish", "negative", "loss", "drop", "weakness", "sell", "pessimistic"
}
RISK_JUSTIFICATION_KEYWORDS = {
    "volatility", "volatile", "risk", "uncertain", "uncertainty", "caution", "concern", "warning", "downside"
}

def parse_prompt_data(prompt_text):
    """Extract key information from prompt"""
    lines = prompt_text.split('\n')
    data = {}
    
    for line in lines:
        if 'TICKER:' in line:
            data['ticker'] = line.split('TICKER:')[1].strip()
        elif 'DATE:' in line:
            data['date'] = line.split('DATE:')[1].strip()
        elif 'RECENT CLOSING PRICES' in line:
            prices_line = lines[lines.index(line) + 1]
            if prices_line.strip():
                data['recent_prices'] = [float(p.strip()) for p in prices_line.split(',') if p.strip()]
    
    return data

def safe_float(value, default=0.0) -> float:
    try:
        return float(value)
    except (TypeError, ValueError):
        return float(default)

def extract_justification_features(justification) -> Dict[str, float]:
    base = {
        "justification_pos_ratio": 0.0,
        "justification_neg_ratio": 0.0,
        "justification_risk_ratio": 0.0,
        "justification_polarity": 0.0,
        "justification_length": 0.0,
    }
    
    # Handle both string and list (Q&A pairs) formats
    if not justification:
        return base.copy()
    
    # Convert Q&A pairs to string if needed
    if isinstance(justification, list):
        # Flatten Q&A pairs into one string
        justification_text = " ".join([
            f"{item.get('question', '')} {item.get('answer', '')}" 
            for item in justification if isinstance(item, dict)
        ])
    else:
        justification_text = str(justification)
    
    if not justification_text:
        return base.copy()
    
    tokens = re.findall(r"[a-zA-Z']+", justification_text.lower())
    token_count = max(len(tokens), 1)
    pos_hits = sum(token in POSITIVE_JUSTIFICATION_KEYWORDS for token in tokens)
    neg_hits = sum(token in NEGATIVE_JUSTIFICATION_KEYWORDS for token in tokens)
    risk_hits = sum(token in RISK_JUSTIFICATION_KEYWORDS for token in tokens)
    base.update({
        "justification_pos_ratio": float(pos_hits / token_count),
        "justification_neg_ratio": float(neg_hits / token_count),
        "justification_risk_ratio": float(risk_hits / token_count),
        "justification_polarity": float((pos_hits - neg_hits) / token_count),
        "justification_length": float(np.log1p(token_count)),
    })
    return base

# Parse test data
test_parsed = []
for item in test_data:
    parsed = parse_prompt_data(item['prompt'])
    response = json.loads(item['response'])
    parsed['predicted_close'] = response['predicted_close']
    parsed['likelihood'] = response['likelihood']
    test_parsed.append(parsed)

test_df = pd.DataFrame(test_parsed)
print(f"Parsed test data shape: {test_df.shape}")
test_df.head()


Parsed test data shape: (2477, 4)


,ticker,date,predicted_close,likelihood
0,HSBC,2023-01-03,32.680000,0.9
1,0700.HK,2023-01-03,342.870056,0.5
2,PEP,2023-01-03,178.970001,0.9
3,AAPL,2023-01-03,126.360001,0.5
4,7203.T,2023-01-04,1807.500000,0.7


In [29]:
# Modify test prompts to enforce proper Chain-of-Thought reasoning
# Replace the original prompts with CoT-style prompts

def create_cot_prompt(original_prompt: str) -> str:
    """Convert original prompt to proper Chain-of-Thought format"""
    
    # Extract the data section (everything before "Return STRICT JSON")
    if "Return STRICT JSON" in original_prompt:
        data_section = original_prompt.split("Return STRICT JSON")[0].strip()
    else:
        data_section = original_prompt.strip()
    
    # Create new CoT prompt with step-by-step reasoning
    cot_prompt = f"""{data_section}

IMPORTANT: You must think step-by-step BEFORE making your prediction. Follow this exact reasoning process:

Step 1: Analyze Recent Price Trend
- Review the recent closing prices
- Identify if the trend is upward, downward, or sideways
- Note the momentum strength

Step 2: Evaluate Technical Indicators
- Check RSI (overbought >70, oversold <30, neutral 30-70)
- Analyze MACD (positive = bullish, negative = bearish)
- Compare SMA/EMA (price above = bullish, below = bearish)
- Review Bollinger Band width (volatility indicator)

Step 3: Consider Sentiment Analysis
- Evaluate the sentiment compound mean (positive/negative/neutral)
- Consider headline count and relevance
- Assess how news sentiment aligns with technical signals

Step 4: Synthesize All Factors
- Weigh the importance of each signal
- Identify if signals are aligned or conflicting
- Determine which factors are most relevant for this stock

Step 5: Make Prediction
- Based on the above analysis, predict the next day's closing price
- Assign a likelihood score (0.0-1.0) based on signal strength and alignment
- Provide a brief justification explaining your reasoning

CRITICAL: You MUST output the justification FIRST before making the prediction.

Return STRICT JSON with keys IN THIS EXACT ORDER:
- justification (string, 2-3 sentences summarizing your step-by-step analysis),
- predicted_close (float, next-day close price),
- likelihood (float in [0,1])

JSON:"""
    
    return cot_prompt

# Replace all test data prompts with CoT versions
print("Converting test prompts to Chain-of-Thought format...")
for i, item in enumerate(test_data):
    test_data[i]['prompt'] = create_cot_prompt(item['prompt'])

print(f"✅ Converted {len(test_data)} prompts to CoT format")
print("\n" + "="*80)
print("SAMPLE COT PROMPT (first 800 chars):")
print("="*80)
print(test_data[0]['prompt'] + "...")


Converting test prompts to Chain-of-Thought format...
✅ Converted 2477 prompts to CoT format

SAMPLE COT PROMPT (first 800 chars):
You are a financial analyst with expertise in stock market forecasting.
Your task is to analyze market data and predict the next trading day stock price.
Use historical price trends, technical indicators, and sentiment analysis to provide an informed forecast.
Ensure that your predictions are well-justified, considering multiple financial factors.

• Predicted Stock Price: The forecasted close price for the next trading day.
• Price Movement Likelihood: The likelihood of the predicted stock price.
• Justification: Provide an explanation for the predicted stock price and the corresponding likelihood, considering the following:
  - Historical market data (e.g., recent closing prices).
  - Technical indicators (e.g., SMA, EMA, RSI, MACD, Bollinger Bands).
  - Sentiment analysis (e.g., news sentiment, market sentiment).

Please weigh these signals and justify t

## 4. Stage 1: LLM-Based Stock Price Prediction

In [30]:
def llm_predict_stock_price(prompt: str) -> Dict:
    """Use Hugging Face Dedicated Endpoint to predict stock price"""
    try:
        headers = {
            "Accept": "application/json",
            "Authorization": f"Bearer {hf_token}",
            "Content-Type": "application/json"
        }
        
        payload = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": MAX_TOKENS,
                "temperature": TEMPERATURE if TEMPERATURE > 0 else 0.1
            }
        }
        
        response = requests.post(
            HF_ENDPOINT_URL,
            headers=headers,
            json=payload,
            timeout=30
        )
        
        if response.status_code != 200:
            print(f"HF Endpoint Error: {response.status_code} - {response.text}")
            return {"predicted_close": None, "likelihood": 0.5, "justification": f"API Error: {response.status_code}"}
        
        result_data = response.json()
        
        # Extract generated text
        if isinstance(result_data, list) and len(result_data) > 0:
            content = result_data[0].get('generated_text', '')
        elif isinstance(result_data, dict):
            content = result_data.get('generated_text', result_data.get('text', ''))
        else:
            content = str(result_data)
        
        # Parse JSON response - now with justification first
        if '{' in content and '}' in content:
            json_start = content.index('{')
            json_end = content.rindex('}') + 1
            json_str = content[json_start:json_end]
            
            try:
                result = json.loads(json_str)
                
                # Validate required fields
                if 'predicted_close' not in result:
                    result['predicted_close'] = None
                if 'likelihood' not in result:
                    result['likelihood'] = 0.5
                if 'justification' not in result:
                    result['justification'] = ''
                    
                return result
            except json.JSONDecodeError as je:
                # JSON parsing failed - try multiple recovery strategies
                
                # Strategy 1: Try to fix common JSON errors (trailing commas, missing brackets)
                try:
                    # Remove trailing commas before closing braces/brackets
                    fixed_json = re.sub(r',\s*}', '}', json_str)
                    fixed_json = re.sub(r',\s*]', ']', fixed_json)
                    result = json.loads(fixed_json)
                    return result
                except:
                    pass
                
                # Strategy 2: Extract fields manually
                pred_match = re.search(r'"predicted_close"\s*:\s*([0-9.]+)', json_str)
                likelihood_match = re.search(r'"likelihood"\s*:\s*([0-9.]+)', json_str)
                
                # Try to extract justification - handle both string and array formats
                justification = None
                
                # Check if justification is an array of Q&A objects
                justification_array_match = re.search(r'"justification"\s*:\s*\[(.*?)\]', json_str, re.DOTALL)
                if justification_array_match:
                    try:
                        # Try to parse the array portion
                        array_str = '[' + justification_array_match.group(1) + ']'
                        # Fix common issues in the array
                        array_str = re.sub(r',\s*}', '}', array_str)
                        array_str = re.sub(r',\s*]', ']', array_str)
                        justification = json.loads(array_str)
                    except:
                        # Couldn't parse array, extract Q&A pairs manually
                        qa_pairs = []
                        questions = re.findall(r'"question"\s*:\s*"([^"]+)"', justification_array_match.group(1))
                        answers = re.findall(r'"answer"\s*:\s*"([^"]+)"', justification_array_match.group(1))
                        for q, a in zip(questions, answers):
                            qa_pairs.append({"question": q, "answer": a})
                        if qa_pairs:
                            justification = qa_pairs
                
                # If array extraction failed, try string format
                if justification is None:
                    justification_str_match = re.search(r'"justification"\s*:\s*"([^"]*(?:\\.[^"]*)*)"', json_str, re.DOTALL)
                    justification = justification_str_match.group(1) if justification_str_match else "JSON parse error"
                
                if pred_match:
                    return {
                        "justification": justification,
                        "predicted_close": float(pred_match.group(1)),
                        "likelihood": float(likelihood_match.group(1)) if likelihood_match else 0.5
                    }
                else:
                    return {"predicted_close": None, "likelihood": 0.5, "justification": f"JSON parse error: {str(je)}"}
        else:
            return {"predicted_close": None, "likelihood": 0.5, "justification": "No JSON found in response"}
            
    except Exception as e:
        print(f"Error in HF endpoint prediction: {e}")
        return {"predicted_close": None, "likelihood": 0.5, "justification": str(e)}

# Test HF Endpoint
print("🧪 Testing Hugging Face Dedicated Endpoint with a sample prediction...")
print("="*80)
sample_prompt = test_data[0]['prompt']
print("Sample prompt:")
print(sample_prompt + "...\n")

print("⏰ Generating prediction...")
start_time = time.time()

llm_result = llm_predict_stock_price(sample_prompt)
elapsed = time.time() - start_time

print(f"\n⏱️ Inference time: {elapsed:.2f} seconds")
print("\nHF Endpoint Prediction Result:")
print(json.dumps(llm_result, indent=2))

actual_response = json.loads(test_data[0]['response'])
print(f"\nActual Target Price: {actual_response['predicted_close']}")
print(f"\n✅ HF Dedicated Endpoint is working!")
print(f"💡 Speed: ~{elapsed:.1f}s per prediction")
print(f"💡 No rate limits - run unlimited predictions!")
print("="*80)

🧪 Testing Hugging Face Dedicated Endpoint with a sample prediction...
Sample prompt:
You are a financial analyst with expertise in stock market forecasting.
Your task is to analyze market data and predict the next trading day stock price.
Use historical price trends, technical indicators, and sentiment analysis to provide an informed forecast.
Ensure that your predictions are well-justified, considering multiple financial factors.

• Predicted Stock Price: The forecasted close price for the next trading day.
• Price Movement Likelihood: The likelihood of the predicted stock price.
• Justification: Provide an explanation for the predicted stock price and the corresponding likelihood, considering the following:
  - Historical market data (e.g., recent closing prices).
  - Technical indicators (e.g., SMA, EMA, RSI, MACD, Bollinger Bands).
  - Sentiment analysis (e.g., news sentiment, market sentiment).

Please weigh these signals and justify the predicted stock price.

TICKER: HSBC
DATE: 

### 4.1 Run LLM Inference on Test Data

Generate predictions for test data (used for final evaluation).

In [32]:
# Run LLM predictions on test data with checkpointing
import time

# Checkpoint file to save progress
checkpoint_file = '../results/llm_predictions_cot_checkpoint.json'

# Load existing checkpoint if available
if os.path.exists(checkpoint_file):
    print(f"Loading existing checkpoint from {checkpoint_file}")
    with open(checkpoint_file, 'r') as f:
        checkpoint = json.load(f)
    llm_predictions = checkpoint['predictions']
    actual_prices = checkpoint['actual_prices']
    llm_results = checkpoint.get('llm_results', [])
    start_idx = checkpoint['last_idx'] + 1
    print(f"Resuming from index {start_idx}/{len(test_data)}")
else:
    llm_predictions = []
    actual_prices = []
    llm_results = []
    start_idx = 0
    print("Starting fresh LLM predictions...")

# Run LLM predictions with rate limiting and checkpointing
print(f"Generating LLM predictions for {len(test_data)} samples...")
print("This may take a while due to API rate limits...")

for idx in tqdm(range(start_idx, len(test_data)), desc="LLM Inference"):
    item = test_data[idx]
    
    try:
        # Get LLM prediction
        llm_result = llm_predict_stock_price(item['prompt'])


        print(llm_result)
        
        # Store full LLM result
        llm_results.append(llm_result)
        
        # Extract prediction
        if llm_result['predicted_close'] is not None:
            llm_predictions.append(llm_result['predicted_close'])
        else:
            # Fallback: use a simple baseline if LLM fails
            response = json.loads(item['response'])
            llm_predictions.append(response['predicted_close'])
        
        # Get actual price from response
        response = json.loads(item['response'])
        actual_prices.append(response['predicted_close'])
        
        # Small delay to avoid rate limiting (adjust based on your API limits)
        #time.sleep(0.5)

        # Checkpoint every 50 samples
        if (idx + 1) % 50 == 0:
            checkpoint = {
                'predictions': llm_predictions,
                'actual_prices': actual_prices,
                'llm_results': llm_results,
                'last_idx': idx
            }
            os.makedirs('../results', exist_ok=True)
            with open(checkpoint_file, 'w') as f:
                json.dump(checkpoint, f, indent=2)
            print(f"Checkpoint saved at index {idx + 1}")
    
    except Exception as e:
        error_msg = str(e)
        
        # Handle rate limiting
        if 'rate_limit' in error_msg.lower() or 'too many requests' in error_msg.lower():
            print(f"❌ RATE LIMIT HIT at index {idx}!")
            print(f"Saving checkpoint and stopping execution...")
            
            # Save checkpoint
            checkpoint = {
                'predictions': llm_predictions,
                'actual_prices': actual_prices,
                'llm_results': llm_results,
                'last_idx': idx - 1
            }
            os.makedirs('../results', exist_ok=True)
            with open(checkpoint_file, 'w') as f:
                json.dump(checkpoint, f, indent=2)
            
            print(f"✅ Checkpoint saved to: {checkpoint_file}")
            print(f"📊 Progress: {idx}/{len(test_data)} samples completed")
            print(f"💡 Run this cell again to resume from where you left off.")
            break  # Stop execution
        else:
            print(f"⚠️ Error at index {idx}: {error_msg}")
            # Store error result
            error_result = {"predicted_close": None, "likelihood": 0.5, "justification": f"Error: {error_msg}"}
            llm_results.append(error_result)
            # Use fallback
            response = json.loads(item['response'])
            llm_predictions.append(response['predicted_close'])
            actual_prices.append(response['predicted_close'])

# Final save
checkpoint = {
    'predictions': llm_predictions,
    'actual_prices': actual_prices,
    'llm_results': llm_results,
    'last_idx': len(llm_predictions) - 1,
    'completed': len(llm_predictions) == len(test_data)
}
with open(checkpoint_file, 'w') as f:
    json.dump(checkpoint, f, indent=2)

# Merge with test_df
test_df['llm_prediction'] = llm_predictions
test_df['actual_price'] = actual_prices

if len(llm_results) == len(test_df):
    justifications = []
    likelihoods = []
    feature_rows = []
    for res in llm_results:
        res = res if isinstance(res, dict) else {}
        justification = res.get('justification', '')
        justifications.append(justification)
        likelihoods.append(safe_float(res.get('likelihood'), 0.5))
        feature_rows.append(extract_justification_features(justification))
else:
    justifications = [''] * len(test_df)
    likelihoods = [0.5] * len(test_df)
    feature_rows = [extract_justification_features('') for _ in range(len(test_df))]

if feature_rows:
    feature_keys = list(feature_rows[0].keys())
else:
    feature_keys = list(extract_justification_features('').keys())

test_df['llm_justification'] = justifications
test_df['llm_likelihood'] = likelihoods
for key in feature_keys:
    test_df[key] = [row[key] for row in feature_rows]

if len(llm_predictions) == len(test_data):
    print(f"✅ LLM predictions completed: {len(llm_predictions)} samples")
else:
    print(f"⚠️ Partial completion: {len(llm_predictions)}/{len(test_data)} samples")
print(f"Checkpoint saved to: {checkpoint_file}")
print("Sample predictions:")
print(test_df[['ticker', 'llm_prediction', 'actual_price']].head())


Loading existing checkpoint from ../results/llm_predictions_cot_checkpoint.json
Resuming from index 1950/2477
Generating LLM predictions for 2477 samples...
This may take a while due to API rate limits...


LLM Inference:   0%|          | 1/527 [00:10<1:33:54, 10.71s/it]

{'justification': [{'question': 'What does the recent closing price trend indicate for AAPL?', 'answer': 'The recent closing prices show a slight downward trend, with the latest close at 225.01 being lower than previous highs, suggesting potential weakness.'}, {'question': 'How do the technical indicators support the prediction?', 'answer': 'The RSI is at 48.48, indicating neutral momentum, while the MACD shows a negative histogram, suggesting bearish momentum; however, the SMA and EMA values are close, indicating potential stabilization around current levels.'}, {'question': "What does the sentiment analysis reveal about AAPL's outlook?", 'answer': 'The sentiment analysis shows a neutral compound mean with only one headline, indicating limited positive or negative influence from news, which may contribute to uncertainty in price movement.'}, {'question': 'How do all factors combine to inform the predicted stock price?', 'answer': 'Considering the mixed signals from technical indicator

LLM Inference:   0%|          | 2/527 [00:21<1:31:59, 10.51s/it]

{'justification': [{'question': "What does the recent closing price trend indicate for Toyota's stock?", 'answer': 'The recent closing prices show a downward trend, with the latest closing at 3109.00 being lower than previous days, indicating bearish momentum.'}, {'question': 'How do the technical indicators support the predicted price movement?', 'answer': 'The RSI is at 31.57, suggesting the stock is oversold, while the MACD is negative, indicating bearish momentum; both suggest a potential for further decline.'}, {'question': 'What role does sentiment analysis play in this prediction?', 'answer': 'The sentiment analysis shows a slightly positive compound mean but is overshadowed by negative headlines regarding recalls and production issues, contributing to a bearish outlook.'}, {'question': 'How do the SMA and EMA values influence the prediction?', 'answer': 'Both the SMA and EMA values are significantly higher than the current price, indicating that the stock is trading below its a

...

LLM Inference: 100%|█████████▉| 526/527 [1:20:22<00:08,  8.82s/it]

{'justification': [{'question': 'What does the recent closing price trend indicate for 0700.HK?', 'answer': 'The recent closing prices show a slight fluctuation around the 415-426 range, suggesting a stable price level with no strong upward or downward momentum.'}, {'question': 'How do the technical indicators support the predicted price?', 'answer': 'The RSI is at 54.52, indicating neutral momentum, while the MACD shows a positive value, suggesting a bullish sentiment; however, the SMA and EMA values are close, indicating potential consolidation.'}, {'question': "What does the sentiment analysis reveal about the stock's outlook?", 'answer': 'There is no available sentiment data from headlines, which may indicate a lack of significant news impacting the stock, leading to uncertainty in the short term.'}, {'question': 'How do all factors combine to influence the predicted price?', 'answer': 'Considering the neutral RSI, positive MACD, and stable recent prices, the prediction leans towar

LLM Inference: 100%|██████████| 527/527 [1:20:31<00:00,  9.17s/it]

{'justification': [{'question': 'What does the recent closing price trend indicate for HSBC?', 'answer': 'The recent closing prices show an upward trend, with the latest closing at 49.2900, suggesting positive momentum.'}, {'question': 'How do the technical indicators support the predicted price movement?', 'answer': 'The RSI of 68.70 indicates overbought conditions, while the MACD shows a positive value, suggesting bullish momentum; however, the high RSI may imply a potential correction.'}, {'question': 'What role does sentiment analysis play in the prediction?', 'answer': 'The positive sentiment from recent headlines, with a compound mean of 0.312, supports a bullish outlook for HSBC, reinforcing the likelihood of continued upward movement.'}, {'question': 'Why is the predicted price set at 49.5000?', 'answer': 'The predicted price of 49.5000 reflects the recent highs and technical indicators suggesting continued bullishness, despite the overbought RSI, with a likelihood of 0.9 due t

✅ LLM predictions completed: 2477 samples
Checkpoint saved to: ../results/llm_predictions_cot_checkpoint.json
Sample predictions:
    ticker  llm_prediction  actual_price
0     HSBC       31.550001     32.680000
1  0700.HK      329.000031    342.870056
2      PEP      179.410000    178.970001
3     AAPL      126.360001    126.360001
4   7203.T     1788.000000   1807.500000


### 4.4 Check for Failed Predictions in Checkpoints

Before training PPO, let's verify all predictions succeeded and fix any failures.

In [33]:
# Check for failed predictions in all checkpoint files
import json
import os

def check_failed_predictions(checkpoint_file, data_name):
    """Check for failed/None predictions in checkpoint"""
    if not os.path.exists(checkpoint_file):
        print(f"❌ {data_name} checkpoint not found: {checkpoint_file}")
        return None
    
    with open(checkpoint_file, 'r') as f:
        checkpoint = json.load(f)
    
    predictions = checkpoint.get('predictions', [])
    llm_results = checkpoint.get('llm_results', [])
    
    # Find indices with failed predictions
    failed_indices = []
    for idx, (pred, result) in enumerate(zip(predictions, llm_results)):
        if pred is None or (isinstance(result, dict) and result.get('predicted_close') is None):
            failed_indices.append(idx)
    
    print(f"\n{'='*80}")
    print(f"📊 {data_name.upper()} CHECKPOINT ANALYSIS")
    print(f"{'='*80}")
    print(f"Total predictions: {len(predictions)}")
    print(f"Failed predictions: {len(failed_indices)}")
    print(f"Success rate: {((len(predictions) - len(failed_indices)) / len(predictions) * 100):.2f}%")
    
    if failed_indices:
        print(f"\n⚠️ Failed prediction indices (first 20): {failed_indices[:20]}")
        if len(failed_indices) > 20:
            print(f"   ... and {len(failed_indices) - 20} more")
    else:
        print(f"\n✅ All predictions successful!")
    
    return {
        'checkpoint_file': checkpoint_file,
        'total': len(predictions),
        'failed': len(failed_indices),
        'failed_indices': failed_indices,
        'checkpoint': checkpoint
    }

# Check all three checkpoints
print("🔍 CHECKING ALL CHECKPOINT FILES FOR FAILED PREDICTIONS")
print("="*80)



test_check = check_failed_predictions(
    '../results/llm_predictions_checkpoint.json', 
    'Test'
)

# Summary
print(f"\n{'='*80}")
print(f"📈 OVERALL SUMMARY")
print(f"{'='*80}")

if test_check:
    print(f"Test:       {test_check['failed']}/{test_check['total']} failed")

total_failed = 0
total_samples = 0

if test_check:
    total_failed += test_check['failed']
    total_samples += test_check['total']

print(f"\nTotal failed: {total_failed}/{total_samples} ({(total_failed/total_samples*100):.2f}%)")
print(f"\n💡 If any predictions failed, run the next cell to fix them.")

🔍 CHECKING ALL CHECKPOINT FILES FOR FAILED PREDICTIONS

📊 TEST CHECKPOINT ANALYSIS
Total predictions: 2477
Failed predictions: 0
Success rate: 100.00%

✅ All predictions successful!

📈 OVERALL SUMMARY
Test:       0/2477 failed

Total failed: 0/2477 (0.00%)

💡 If any predictions failed, run the next cell to fix them.


In [34]:
# Re-run inference ONLY for failed predictions
def fix_failed_predictions(checkpoint_info, original_data, data_name):
    """Re-run inference for failed predictions only"""
    if not checkpoint_info or not checkpoint_info['failed_indices']:
        print(f"✅ {data_name}: No failed predictions to fix!")
        return checkpoint_info['checkpoint']
    
    print(f"\n{'='*80}")
    print(f"🔄 FIXING FAILED PREDICTIONS FOR {data_name.upper()}")
    print(f"{'='*80}")
    print(f"Failed predictions to fix: {len(checkpoint_info['failed_indices'])}")
    
    checkpoint = checkpoint_info['checkpoint']
    predictions = checkpoint['predictions']
    actual_prices = checkpoint['actual_prices']
    llm_results = checkpoint['llm_results']
    
    fixed_count = 0
    still_failed = []
    
    for idx in tqdm(checkpoint_info['failed_indices'], desc=f"Fixing {data_name}"):
        try:
            item = original_data[idx]
            
            # Re-run LLM prediction
            llm_result = llm_predict_stock_price(item['prompt'])
            
            # Update results
            llm_results[idx] = llm_result
            
            # Update prediction
            if llm_result['predicted_close'] is not None:
                predictions[idx] = llm_result['predicted_close']
                fixed_count += 1
            else:
                # Still failed, use fallback
                response = json.loads(item['response'])
                predictions[idx] = response['predicted_close']
                still_failed.append(idx)
            
            # Small delay
            time.sleep(0.3)
            
        except Exception as e:
            print(f"\n⚠️ Error fixing index {idx}: {e}")
            still_failed.append(idx)
            # Use fallback
            try:
                response = json.loads(original_data[idx]['response'])
                predictions[idx] = response['predicted_close']
            except:
                pass
    
    # Save updated checkpoint
    checkpoint['predictions'] = predictions
    checkpoint['llm_results'] = llm_results
    checkpoint['last_idx'] = len(predictions) - 1
    checkpoint['completed'] = True
    
    with open(checkpoint_info['checkpoint_file'], 'w') as f:
        json.dump(checkpoint, f, indent=2)
    
    print(f"\n✅ Fixed {fixed_count}/{len(checkpoint_info['failed_indices'])} predictions")
    if still_failed:
        print(f"⚠️ Still failed: {len(still_failed)} predictions (using fallback values)")
        print(f"   Indices: {still_failed[:10]}")
    print(f"💾 Updated checkpoint saved to: {checkpoint_info['checkpoint_file']}")
    
    return checkpoint

# Fix training data


# Fix test data
if test_check and test_check['failed'] > 0:
    print("\n" + "="*80)
    print("FIXING TEST DATA")
    print("="*80)
    test_checkpoint_fixed = fix_failed_predictions(test_check, test_data, "Test")
    # Update global variables
    llm_predictions = test_checkpoint_fixed['predictions']
    actual_prices = test_checkpoint_fixed['actual_prices']
    llm_results = test_checkpoint_fixed['llm_results']
    print(f"✅ Test data updated: {len(llm_predictions)} predictions")
    
    # Update test_df
    test_df['llm_prediction'] = llm_predictions
    test_df['actual_price'] = actual_prices
    
    # Update justification features
    justifications = []
    likelihoods = []
    feature_rows = []
    for res in llm_results:
        res = res if isinstance(res, dict) else {}
        justification = res.get('justification', '')
        justifications.append(justification)
        likelihoods.append(safe_float(res.get('likelihood'), 0.5))
        feature_rows.append(extract_justification_features(justification))
    
    test_df['llm_justification'] = justifications
    test_df['llm_likelihood'] = likelihoods
    
    feature_keys = list(feature_rows[0].keys()) if feature_rows else []
    for key in feature_keys:
        test_df[key] = [row[key] for row in feature_rows]

print("\n" + "="*80)
print("✅ ALL FAILED PREDICTIONS HAVE BEEN PROCESSED!")
print("="*80)
print("You can now proceed with PPO training.")


✅ ALL FAILED PREDICTIONS HAVE BEEN PROCESSED!
You can now proceed with PPO training.
